In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir /content/drive/My\ Drive/Submissions

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [49.2 kB]
Fetched 334 kB in 2s (187 kB/s)
Reading package lists... Done
W: Skipping acquire of con

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up Chrome options for headless mode
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

chrome_options.add_experimental_option('prefs', {
    "download.default_directory": '/content/drive/My Drive/Submissions/',  # Change this to your desired download directory
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True  # This will prevent Chrome from opening the PDF in the browser
})

In [ ]:
links = []

In [ ]:
def go_to_next_page():
    next_button = driver.find_element(By.XPATH, '//button[@title="Next"]')
    next_button.click()
    time.sleep(4)  # Wait for the page to load

In [ ]:
def scrape_submissions(url):
    # Wait for the content to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "flex.items-center"))
    )

    #submission_items = driver.find_elements(By.CSS_SELECTOR,
#'a.no-underline.flex.text-grey-800.cursor-pointer.block.p-3.border-b.last\\:border-b-0.border-grey-300\\/50.bg-grey-500.bg-opacity-5.hover\\:bg-grey-500.hover\\:bg-opacity-10')

    submission_items = driver.find_elements(By.XPATH, '//a[contains(@href, "/supporting-responsible-ai/submission/view/")]')
    #print(submission_items)

    for item in submission_items:
        href = item.get_attribute('href')
        links.append(href)

In [ ]:
# URL of the page to scrape
url = "https://consult.industry.gov.au/supporting-responsible-ai/submission/list"

# Set up the WebDriver (make sure you have the appropriate driver installed)
driver = webdriver.Chrome(options=chrome_options)  # Use Chrome in headless mode
# Navigate to the page
driver.get(url)

for _ in range(18):  # Adjust the range as needed
    scrape_submissions(url)
    print(f"Scraped page {_}")
    go_to_next_page()

driver.quit()

Scraped page 0
Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
Scraped page 16
Scraped page 17


In [ ]:
len(links)

422

In [ ]:
first = []

def scrape_submissions(url):
    # Wait for the content to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "flex.items-center"))
    )

    #submission_items = driver.find_elements(By.CSS_SELECTOR, 'a.no-underline.flex.text-grey-800.cursor-pointer.block.p-3.border-b.last\\:border-b-0.border-grey-300\\/50.bg-grey-500.bg-opacity-5.hover\\:bg-grey-500.hover\\:bg-opacity-10')

    #submission_items = driver.find_elements(By.XPATH, '//a[contains(@href, "/supporting-responsible-ai/submission/view/")]')
    submission_items = driver.find_elements(By.CSS_SELECTOR, 'a.no-underline.flex.text-grey-800.cursor-pointer.block.p-3.border-b.last\\:border-b-0.border-grey-300\\/50.hover\\:bg-grey-500.hover\\:bg-opacity-10')

    #submission_items = driver.find_elements(By.XPATH, '//a[contains(@class, "no-underline flex text-grey-800 cursor-pointer block p-3 border-b last:border-b-0 border-grey-300/50 bg-grey-500 bg-opacity-5 hover:bg-grey-500 hover:bg-opacity-10") or contains(@class, "no-underline flex text-grey-800 cursor-pointer block p-3 border-b last:border-b-0 border-grey-300/50 hover:bg-grey-500 hover:bg-opacity-10")]')
    #print(submission_items)

    for item in submission_items:
        href = item.get_attribute('href')
        first.append(href)


In [ ]:
# URL of the page to scrape
url = "https://consult.industry.gov.au/supporting-responsible-ai/submission/list"

# Set up the WebDriver (make sure you have the appropriate driver installed)
driver = webdriver.Chrome(options=chrome_options)  # Use Chrome in headless mode
# Navigate to the page
driver.get(url)

scrape_submissions(url)

driver.quit()

In [ ]:
links = first + links

In [ ]:
link = pd.DataFrame(links)

# Save the DataFrame to a CSV file
link.to_csv('submission_links.csv', index=False)

from google.colab import files

files.download('submission_links.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

links = pd.read_csv('/content/drive/My Drive/submission_links.csv')
print(links)

                                                     0
0    https://consult.industry.gov.au/supporting-res...
1    https://consult.industry.gov.au/supporting-res...
2    https://consult.industry.gov.au/supporting-res...
3    https://consult.industry.gov.au/supporting-res...
4    https://consult.industry.gov.au/supporting-res...
..                                                 ...
442  https://consult.industry.gov.au/supporting-res...
443  https://consult.industry.gov.au/supporting-res...
444  https://consult.industry.gov.au/supporting-res...
445  https://consult.industry.gov.au/supporting-res...
446  https://consult.industry.gov.au/supporting-res...

[447 rows x 1 columns]


In [ ]:
links['0']

0      https://consult.industry.gov.au/supporting-res...
1      https://consult.industry.gov.au/supporting-res...
2      https://consult.industry.gov.au/supporting-res...
3      https://consult.industry.gov.au/supporting-res...
4      https://consult.industry.gov.au/supporting-res...
                             ...                        
442    https://consult.industry.gov.au/supporting-res...
443    https://consult.industry.gov.au/supporting-res...
444    https://consult.industry.gov.au/supporting-res...
445    https://consult.industry.gov.au/supporting-res...
446    https://consult.industry.gov.au/supporting-res...
Name: 0, Length: 447, dtype: object

In [ ]:
submissions = []

for link in links['0']:
  print(link)
  driver = webdriver.Chrome(options=chrome_options)  # Use Chrome in headless mode

  driver.get(link)

  try:
    transcript_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "tiptap.ProseMirror.border-0.break-word.outline-0"))
    )

    #published_name_element = WebDriverWait(driver, 10).until(
    #EC.presence_of_element_located((By.XPATH, "//div[contains(text(), 'Published name')]/following-sibling::div")))
    published_name_element = driver.find_element(By.CSS_SELECTOR, 'div.font-bold.text-xl.mr-2')

    # Extract the text from the element
    published_name = published_name_element.text
    #print(published_name)

    transcript_text = transcript_element.text
    #print(transcript_text)

    file_name = published_name + '.txt'
    file_path = '/content/drive/My Drive/Submissions/' + file_name

    with open(file_path, 'w') as file:
      file.write(transcript_text)

    submissions.append({
        'name': published_name,
        'transcription': file_name
    })
  except Exception as e:
    print(f"Failed to extract text: {e}")

    try:
      pdf_link = driver.find_element(By.CSS_SELECTOR, 'a.flex.items-center.truncate.whitespace-nowrap.text-sky-400.underline.hover\\:no-underline.hover\\:underline')
      pdf_url = pdf_link.get_attribute('href')
      driver.get(pdf_url)

      # Wait for the download to complete
      time.sleep(5)  # Adjust the sleep time as needed

    except Exception as e:
          print(f"Failed to download PDF: {e}")

  driver.quit()

https://consult.industry.gov.au/supporting-responsible-ai/submission/view/510
https://consult.industry.gov.au/supporting-responsible-ai/submission/view/507
https://consult.industry.gov.au/supporting-responsible-ai/submission/view/506
https://consult.industry.gov.au/supporting-responsible-ai/submission/view/505
Failed to extract text: Message: 
Stacktrace:
#0 0x5b20d77166ba <unknown>
#1 0x5b20d73e6730 <unknown>
#2 0x5b20d7435d9b <unknown>
#3 0x5b20d7436081 <unknown>
#4 0x5b20d7479cd4 <unknown>
#5 0x5b20d7458a7d <unknown>
#6 0x5b20d74771ca <unknown>
#7 0x5b20d74587f3 <unknown>
#8 0x5b20d7428ec9 <unknown>
#9 0x5b20d742991e <unknown>
#10 0x5b20d76dc9eb <unknown>
#11 0x5b20d76e0972 <unknown>
#12 0x5b20d76c9e15 <unknown>
#13 0x5b20d76e1502 <unknown>
#14 0x5b20d76aed2f <unknown>
#15 0x5b20d7705578 <unknown>
#16 0x5b20d7705750 <unknown>
#17 0x5b20d771548c <unknown>
#18 0x7e4fc5b6aac3 <unknown>

https://consult.industry.gov.au/supporting-responsible-ai/submission/view/504
https://consult.indust

In [ ]:
submissions

[{'name': 'Australian Federal Police',
  'transcription': 'Australian Federal Police.txt'},
 {'name': 'Professor Nicole Gillespie',
  'transcription': 'Professor Nicole Gillespie.txt'},
 {'name': 'Office of the Australian Information Commissioner (OAIC)',
  'transcription': 'Office of the Australian Information Commissioner (OAIC).txt'},
 {'name': 'Law Council of Australia',
  'transcription': 'Law Council of Australia.txt'},
 {'name': 'Digital Industry Group Inc (DIGI)',
  'transcription': 'Digital Industry Group Inc (DIGI).txt'},
 {'name': 'Shopping Centre Council of Australia',
  'transcription': 'Shopping Centre Council of Australia.txt'},
 {'name': 'Department of Health and Aged Care',
  'transcription': 'Department of Health and Aged Care.txt'},
 {'name': 'Commonwealth Bank of Australia',
  'transcription': 'Commonwealth Bank of Australia.txt'},
 {'name': 'IAB Australia', 'transcription': 'IAB Australia.txt'},
 {'name': 'Australian Banking Association',
  'transcription': 'Austra

In [ ]:
df = pd.DataFrame(submissions)

# Save the DataFrame to a CSV file
df.to_csv('submissions.csv', index=False)

from google.colab import files

files.download('submissions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>